# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '4'))
    os.environ['BATCH_SIZE'] = '32'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 4 max_iteration = 5 model_type = base *********
n              iter (since)      4 loss (since)       4 acc (since) 4 acc (dev) (since)      dt
1             0.000 (0.000)       2.592 (2.592)       0.594 (0.594)       0.000 (0.000)  16.9 s
2             0.000 (0.000)       1.997 (1.402)       0.609 (0.625)       0.000 (0.000)  30.4 s
4             0.000 (0.000)       1.396 (0.796)       0.703 (0.797)       0.000 (0.000)  57.7 s
8             0.000 (0.000)       1.081 (0.765)       0.730 (0.758)       0.000 (0.000)  1.74 m
16            0.000 (0.000)       0.895 (0.709)       0.766 (0.801)       0.000 (0.000)  3.32 m
32            0.000 (0.000)       0.815 (0.736)       0.767 (0.768)       0.000 (0.000)  6.27 m
64            0.000 (0.000)       0.761 (0.707)       0.775 (0.784)       0.000 (0.000)  12.9 m
128           0.000 (0.000)       0.748 (0.734)       0.774 (0.773)       0.000 (0.000)  25.5 m
256           0.000 (0.000)       0.708 (0.659)       0.783 (0.795)  

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '32'
    os.environ['LEARN_BATCH_SIZE'] = '16'
    os.environ['STEP1_ITER'] = '0_augment4'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 1 max_iteration = 5 model_type = base *********
n              iter (since)      4 loss (since)       4 acc (since) 4 acc (dev) (since)      nsamps (since)      dt
1             0.000 (0.000)       0.560 (0.560)       0.719 (0.719)       0.000 (0.000)      57.219 (57.219)  1.16 m
2             0.000 (0.000)       0.594 (0.629)       0.750 (0.781)       0.000 (0.000)      30.656 (4.094)  1.63 m
4             0.000 (0.000)       0.526 (0.457)       0.781 (0.812)       0.000 (0.000)      17.719 (4.781)  2.48 m
8             0.000 (0.000)       0.536 (0.546)       0.781 (0.781)       0.000 (0.000)      11.125 (4.531)  4.44 m
16            0.000 (0.000)       0.522 (0.507)       0.791 (0.801)       0.000 (0.000)       8.256 (5.387)  8.53 m
32            0.000 (0.000)       0.523 (0.524)       0.789 (0.787)       0.000 (0.000)       6.785 (5.314)    17 m
64            0.000 (0.000)       0.522 (0.519)       0.792 (0.799)       0.812 (0.812)       6.043 (4.458)  28.2 m
80  

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission_probensemble(*, nvoters, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNewsCat import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.load_adapter(f'User_keq{k}_t5base_step2_iter{step2_iter}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm", choices=dev.choices)
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'User_keq{k}_t5base_step2_iter{step2_iter}')

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} nvoters {nvoters} ***')
    
    with ProgressPrinter(f'{k} acc (dev)') as printer:
        devgolds, testgolds = [], []
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                predicts = torch.cat(inner_batch(func = taskllm.predict,
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []),)
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ predicts[a:b,:].logsumexp(dim=0, keepdim=True).argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                if isdev:
                    targets = [ dev.choices.index(label) for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    acc = (guesses == targets).float().mean().item()
                else:
                    acc = None

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': dev.choices[guess] })

            printer.addobs(acc)

        printer.print()
        printer.autoprint = False

        for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
            with open(f'lamp2u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}_nvoters{nvoters}.json', 'w') as jsonfile:
                json.dump({ 'task': 'LaMP_2', 'golds': golds }, jsonfile)
            
for nvoters in [1, 3, 5, 7]:
    prepare_submission_probensemble(k=4, step1_iter='0_augment4', step2_iter='2_augment1', nvoters=nvoters)

*** step1_iter: 0_augment4 step2_iter: 2_augment1 nvoters 1 ***
n       4 acc (dev) (since)      dt
1             0.500 (0.500)  13.4 s
2             0.562 (0.625)  26.7 s
4             0.719 (0.875)    49 s
8             0.766 (0.812)  1.61 m
16            0.820 (0.875)  3.12 m
32            0.844 (0.867)   6.3 m
64            0.820 (0.797)  12.7 m
128           0.824 (0.828)  25.6 m
256           0.826 (0.875)  54.8 m
*** step1_iter: 0_augment4 step2_iter: 2_augment1 nvoters 3 ***
n       4 acc (dev) (since)      dt
1             0.500 (0.500)  14.1 s
2             0.625 (0.750)  28.5 s
4             0.750 (0.875)  52.5 s
8             0.781 (0.812)   1.7 m
16            0.820 (0.859)  3.26 m
32            0.840 (0.859)  6.51 m
64            0.812 (0.785)    13 m
128           0.819 (0.826)  26.3 m
256           0.821 (0.875)    56 m
*** step1_iter: 0_augment4 step2_iter: 2_augment1 nvoters 5 ***
n       4 acc (dev) (since)      dt
1             0.500 (0.500)  14.3 s
2             0.